In [ ]:
from mlrepricer import listener, redisdb, updateprice, minmax, asintosku, helper

In [ ]:
#redisdb.server_start()

In [14]:
asintosku.AsintoSku().start()

Starting Thread-7, this may take 2 mins


# listener demon
to get messages from mws queues and dump them into a filesystem

In [ ]:
listener.Listener().start()

In [ ]:
updateprice.Updater().start()

In [11]:
def get_report(report_name):
    """Call reports easy."""
    report_api = mws.apis.Reports(**helper.mwscred)
    request_response = report_api.request_report(report_name)
    report_id = helper.auto_get_report_id(request_response)
    if report_id:
        report = report_api.get_report(report_id).parsed
    return report

def read_filter(report):
    i = pd.read_csv(io.StringIO(report), sep='\t', decimal='.',
                    thousands=None, dayfirst=True)
    i = i[i.loc[:, 'item-condition'] == 11]
    d = i.loc[:, ['seller-sku', 'asin1', 'fulfillment-channel']]
    d['county'] = 1
    return d.groupby(['asin1', 'fulfillment-channel']).agg(
        {'seller-sku': 'first', 'county': 'count'})


def deduplicate(listingperasin):
    """Remove duplicates."""
    duplicates = listingperasin[listingperasin.loc[:, 'county'] > 1]
    if len(duplicates) >= 1:
        print(f'you may want to delete one listing per asin {duplicates}')

    # those we return
    return listingperasin[listingperasin.county == 1].reset_index()


def reorganize(nocollusion):
    """We do some final cleanup and reorganize columns."""
    # When fulfillment-channel is DEFAULT its seller fulfilled
    nocollusion['isprime'] = np.where(
        nocollusion['fulfillment-channel'] == 'DEFAULT', 0, 1)
    nocollusion.drop(['fulfillment-channel', 'county'], axis=1, inplace=True)
    nocollusion.rename(
        {'asin1': 'asin', 'seller-sku': 'seller_sku'}, axis=1, inplace=True)
    helper.dump_dataframe(nocollusion, 'asintosku')


In [9]:
import io
import mws
from mws.apis.reports import ReportType
import numpy as np
import pandas as pd
import threading

report = get_report(ReportType.ACTIVE_LISTINGS.value)

In [13]:
# store the report
reorganize(deduplicate(read_filter(report)))

you may want to delete one listing per asin                                               seller-sku  county
asin1      fulfillment-channel                                  
B075NJNLJT DEFAULT                big_ice_pop_aroma_30ml       2
B075NJTZ9B DEFAULT                  big_jelly_aroma_30ml       2
B075NK775D DEFAULT                  big_zingy_aroma_30ml       2
B075NKBZFM DEFAULT              big_fruitypop_aroma_30ml       2
B075NNCZRR DEFAULT                  Big_loved30ml_LeTree       2
B075NQJ16Z DEFAULT              big_lollipops_aroma_30ml       2
B078CQ2WFR DEFAULT                  big_fcapb_aroma_30ml       2


In [ ]:
minmax.load_csv()

In [10]:
report

'item-name\titem-description\tlisting-id\tseller-sku\tprice\tquantity\topen-date\timage-url\titem-is-marketplace\tproduct-id-type\tzshop-shipping-fee\titem-note\titem-condition\tzshop-category1\tzshop-browse-path\tzshop-storefront-feature\tasin1\tasin2\tasin3\twill-ship-internationally\texpedited-shipping\tzshop-boldface\tproduct-id\tbid-for-featured-placement\tadd-delete\tpending-quantity\tfulfillment-channel\tBusiness Price\tQuantity Price Type\tQuantity Lower Bound 1\tQuantity Price 1\tQuantity Lower Bound 2\tQuantity Price 2\tQuantity Lower Bound 3\tQuantity Price 3\tQuantity Lower Bound 4\tQuantity Price 4\tQuantity Lower Bound 5\tQuantity Price 5\tmerchant-shipping-group\neVod und eVod 2 USB-Ladekabel für E-Zigaretten - original Kangertech\tLadegerät für den Kangertech Evod Verdampfer und alle Akkus mit Emow-Anschluss\t1213UMZ0DVW\t16003003501\t9.79\t29\t13/12/2018 23:28:44 MET\t\ty\t1\t\t\t11\t\t\t\tB00JLA94V6\t\t\t\t\t\tB00JLA94V6\t\t\t0\tDEFAULT\t\t\t\t\t\t\t\t\t\t\t\t\tStanda